In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [21]:
import sys
sys.path.append('../src')

from spark import SparkSessionFactory, write_csv, read_csv, column_values, train_test_split, PageSet, get_columns, get_rows

from data import MovieLensDataSource
from util import create_path, remove_dir

import pyspark.sql.types as t
import pyspark.sql as s
import pyspark.sql.functions as f

# Recommedations dataset building

This is the first step of the process to train a model that predict an user rating to a given movie. Before all, we needs to build a table of features from a raw dataset. This table must be contain next columns:

**feature table columns**:
* **user_id**: This is a categorical column because this has discrete values one of for each user in raw dataset.
* **movie_id**: Like user_id column, this has discrete values one for each movie in raw dataset. 
* **one column for each movie genre**
  * For example: accion_genre, drama_gender, etc... are possible columns. Each one takes a discrete value 1 or 0.
  * Why is there one column by genre? Because, each movie could have one or more genres. 
* **rating**: Score that a user(user_id) gives to a movie(movie_id).

Finally, we have a table with discreate column values, they are also called categorical columns, because this columns only take one value from a finite list of possible values. An example of this table must be:

| user_id | movie_id | accion_genre | drama_genre | rating |
| --- | --- | --- | --- | --- |
| 1 |  1 | 1 | 0 | 5 |
| 1 |  2 | 0 | 1 | 1 |
| 1 |  3 | 1 | 1 | 3 |

* User 1 rated the movie 1 with a score of 5 points, because he loves accion movies.
* User 1 rated the movie 2 with a score of 1 points, because he is not interested to drama movies.
* User 1 rated the movie 3 with a score of 2 points, given that he loves accion but is not interested to drama.
* action_genre/drama_genre is a categorical column because only could take 1 or 0 values.
* user_id is categorical column because only could take any of user ids(Exist a discrte number of user ids).
* movie_id is categorical column because only could take any of movie ids(Exist a discrte number of user ids).

## Building out features table

**Step 1**: Let's to start create a new spark session.

In [3]:
session = SparkSessionFactory.create()
session

**Step 2**: Select a raw dataset size. For this recommendation examples we have 3 raw datasets each with distinct number of examples. Sometimes we need to begins with a tiny dataset to try to create a model that overfit to these data. This tell us that the model understand input patterns although this does not generalize yet but the model understand input patterns. The next step is maybe use more data and try to find the best hiperparameters so that the model generalize as well as possible.

In [4]:
print('Dataset sizes:', MovieLensDataSource.sizes())

Dataset sizes: ['ml-latest-small', 'ml-25m', 'ml-latest']


In [5]:
# dataset_size = 'ml-latest' # 250MB
# dataset_size = 'ml-25m' # 25MB
dataset_size = 'ml-latest-small' # 1MB

### Download dataset

**Step 3**: Fisrt let's check dataset files.

Check: [Movie lens datasets](https://grouplens.org/datasets/movielens/)

In [6]:
ds = MovieLensDataSource(size = dataset_size)

In [7]:
ds.file_paths()

['/home/adrian/.keras/datasets/ml-latest-small/ratings.csv',
 '/home/adrian/.keras/datasets/ml-latest-small/tags.csv',
 '/home/adrian/.keras/datasets/ml-latest-small/links.csv',
 '/home/adrian/.keras/datasets/ml-latest-small/movies.csv']

**Step 4**: Load ratings and movies files to a dataset for each one. 

In [8]:
ratings = read_csv(session, ds.file_paths()[0])
movies = read_csv(session, ds.file_paths()[3])

**Step 5**: Let's see ratings and movies schemas.

In [9]:
ratings.show(5, truncate=False)
ratings.printSchema()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|1     |1      |4.0   |964982703|
|1     |3      |4.0   |964981247|
|1     |6      |4.0   |964982224|
|1     |47     |5.0   |964983815|
|1     |50     |5.0   |964982931|
+------+-------+------+---------+
only showing top 5 rows

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [10]:
movies.show(5, truncate=False)
movies.printSchema()

+-------+----------------------------------+-------------------------------------------+
|movieId|title                             |genres                                     |
+-------+----------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                  |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                    |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)           |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)          |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)|Comedy                                     |
+-------+----------------------------------+-------------------------------------------+
only showing top 5 rows

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



**Step 6**: Left join both tables by movieId and select rating score, userId, movieId and genres columns.

In [11]:
ratings = ratings \
    .join(movies, on='movieId', how="left") \
    .select(
        f.col('rating'),    
        f.col('userId').alias('user_id'), 
        f.col('movieId').alias('movie_id'), 
        f.col('genres')
    )

ratings.show(5, truncate=False)

+------+-------+--------+-------------------------------------------+
|rating|user_id|movie_id|genres                                     |
+------+-------+--------+-------------------------------------------+
|4.0   |1      |1       |Adventure|Animation|Children|Comedy|Fantasy|
|4.0   |1      |3       |Comedy|Romance                             |
|4.0   |1      |6       |Action|Crime|Thriller                      |
|5.0   |1      |47      |Mystery|Thriller                           |
|5.0   |1      |50      |Crime|Mystery|Thriller                     |
+------+-------+--------+-------------------------------------------+
only showing top 5 rows



**Step 7**: Group by movie genres to show all genres combinations.

In [12]:
genres_count = ratings \
    .groupBy("genres") \
    .count() \
    .orderBy(f.column('count').desc())

genres_count.show(10, truncate=False)

+-------------------------+-----+
|genres                   |count|
+-------------------------+-----+
|Comedy                   |7196 |
|Drama                    |6291 |
|Comedy|Romance           |3967 |
|Comedy|Drama|Romance     |3000 |
|Comedy|Drama             |2851 |
|Drama|Romance            |2838 |
|Action|Adventure|Sci-Fi  |2361 |
|Crime|Drama              |2315 |
|Action|Crime|Thriller    |1554 |
|Action|Adventure|Thriller|1455 |
+-------------------------+-----+
only showing top 10 rows



**Step 8**: Create genres column names:

In [13]:
def normalize(value): return value.lower().replace('-','_')

In [14]:
genres_columns = genres_count \
        .select('genres') \
        .rdd \
        .flatMap(lambda it: it[0].split('|')) \
        .distinct() \
        .map(lambda it: (it, 'gen_none' if '(' in it else f'gen_{normalize(it)}')) \
        .collect()

genres_columns

[('Romance', 'gen_romance'),
 ('Horror', 'gen_horror'),
 ('Sci-Fi', 'gen_sci_fi'),
 ('Comedy', 'gen_comedy'),
 ('Children', 'gen_children'),
 ('Documentary', 'gen_documentary'),
 ('Film-Noir', 'gen_film_noir'),
 ('Action', 'gen_action'),
 ('Fantasy', 'gen_fantasy'),
 ('Drama', 'gen_drama'),
 ('Thriller', 'gen_thriller'),
 ('(no genres listed)', 'gen_none'),
 ('Crime', 'gen_crime'),
 ('War', 'gen_war'),
 ('Musical', 'gen_musical'),
 ('Adventure', 'gen_adventure'),
 ('Mystery', 'gen_mystery'),
 ('Western', 'gen_western'),
 ('Animation', 'gen_animation'),
 ('IMAX', 'gen_imax')]

**Step 9**: create features table.

In [15]:
for (genre, col_name) in genres_columns:
    ratings = ratings.withColumn(col_name, ratings.genres.contains(genre).cast('integer'))

ratings = ratings.drop('genres')
    
ratings.show(5, truncate=False)

+------+-------+--------+-----------+----------+----------+----------+------------+---------------+-------------+----------+-----------+---------+------------+--------+---------+-------+-----------+-------------+-----------+-----------+-------------+--------+
|rating|user_id|movie_id|gen_romance|gen_horror|gen_sci_fi|gen_comedy|gen_children|gen_documentary|gen_film_noir|gen_action|gen_fantasy|gen_drama|gen_thriller|gen_none|gen_crime|gen_war|gen_musical|gen_adventure|gen_mystery|gen_western|gen_animation|gen_imax|
+------+-------+--------+-----------+----------+----------+----------+------------+---------------+-------------+----------+-----------+---------+------------+--------+---------+-------+-----------+-------------+-----------+-----------+-------------+--------+
|4.0   |1      |1       |0          |0         |0         |1         |1           |0              |0            |0         |1          |0        |0           |0       |0        |0      |0          |1            |0       

In [16]:
from sklearn.preprocessing import LabelEncoder

class Sequencer:
    def __init__(self):
        self.sequence = -1
        self.mapping = {}

    def __call__(self, value):
        if value in self.mapping:
            return self.mapping[value]

        self.sequence += 1
        self.mapping[value] = self.sequence
        return self.sequence

user_seq = f.udf(Sequencer(), t.IntegerType())
movie_seq = f.udf(Sequencer(), t.IntegerType())

ratings = ratings \
    .withColumn('user_seq', user_seq(f.col('user_id'))) \
    .withColumn('movie_seq', movie_seq(f.col('movie_id')))

ratings.select(f.col('user_seq'), f.col('movie_seq')) \
    .show(5, truncate=False)

+--------+---------+
|user_seq|movie_seq|
+--------+---------+
|0       |0        |
|0       |1        |
|0       |2        |
|0       |3        |
|0       |4        |
+--------+---------+
only showing top 5 rows



**Step 10**: Write table a file.

In [17]:
TEMP_PATH = './temp'
DATASET_PATH = f'{TEMP_PATH}/dataset'

In [22]:
remove_dir(DATASET_PATH)
create_path(DATASET_PATH)

'./temp/dataset'

In [23]:
write_csv(ratings, DATASET_PATH)

In [24]:
session.stop()